In [ ]:
import requests
import json
import pandas as pd

def get_videos(api_key, channel_id):
    base_video_url = 'https://www.youtube.com/watch?v='
    base_search_url = 'https://www.googleapis.com/youtube/v3/search?'

    first_url = base_search_url+'key={}&channelId={}&part=snippet,id&order=date&maxResults=25'.format(api_key, channel_id)

    video_links = []
    url = first_url
    while True:
        inp = requests.get(url)
        resp = json.loads(inp.text)
        for i in resp['items']:
            if i['id']['kind'] == "youtube#video":
                video_links.append(base_video_url + i['id']['videoId'])

        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break
    return video_links

api_key = 'API 키'
channel_id = '채널 ID'

video_links = get_videos(api_key, channel_id)
df = pd.DataFrame(video_links, columns=['URL'])

In [ ]:
from pytube import YouTube
import pandas as pd

def video_info(df):

    old_name = ''

    i = 1

    li = df["URL"].values

    for url in li:

        yt = YouTube(url)

        if old_name == yt.author:
            i += 1

        else:
            i = 1

        if yt.length <= 60:
            continue

        index = yt.author + '-' + str(i)
        data = {'제목': yt.title, '길이': yt.length, '게시날짜': yt.publish_date,
                '조회수': yt.views, '썸네일 주소': yt.thumbnail_url}

        v_info.loc[index] = data

        old_name = yt.author

In [ ]:
v_info = pd.DataFrame(columns=['제목', '길이', '게시날짜', '조회수', '썸네일 주소'])

video_info(df)

In [ ]:
v_info.to_csv('고몽.csv')